# The Prompt Generator Pattern - AI as a Prompt Engineer

Hello everyone. Today, we're exploring a fascinating "meta" technique where we use an AI to help us do our job as prompt engineers. So far, we have been carefully crafting prompts. But what if we could use the AI itself to build better prompts for us?

This is the **Prompt Generator Pattern**. The core idea is to use a powerful, state-of-the-art model to act as your expert prompt engineering partner. Its task is not to solve the final problem, but to **generate a high-quality, optimized prompt** that you can then use for your actual task.

This is especially useful when you want to use a cheaper, faster model for a repetitive production task but need the prompt to be perfectly optimized for reliability.

## Helper Functions

First, let's set up our standard helper functions.

In [2]:
import litellm
from IPython.display import display, Markdown
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

MODEL_NAME = "openai/gpt-4o-mini"
MAX_TOKENS_DEFAULT = 500

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    if "gpt-5" in model:
        kwargs["max_completion_tokens"] = max_tokens
    else:
        kwargs["max_tokens"] = max_tokens
        
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    response = litellm.completion(
        model=model,
        messages=parsed_messages,
        **kwargs
    )

    return response.choices[0].message.content

print("Setup complete. Helper functions and code context are ready.")

Setup complete. Helper functions and code context are ready.


## The Master Prompt (Generating a Prompt)

Our first step is to craft a "master prompt." This prompt doesn't solve the final task; it *describes the prompt we want*. We will go ahead and implement a conversation flow to interactively craft our prompt. (You can also just do this exercise in ChatGPT, Claude Desktop, or any other AI application).

In [12]:
master_prompt = [
    {
        "role": "system",
        "content": dedent("""
        You are an expert prompt engineer, and your task is to create a high-quality,
        optimized system prompt based on a user's specification.

        ## Process:

        1. Start by asking me to provide the task I want you to generate a prompt for.
        2. Once I provide the task, ask me questions to clarify any doubts or missing information.
        3. Once you have the necessary information, create a detailed and effective system prompt that
            I can use with an AI system to tackle the task at hand.
        4. When I write "GENERATE", you will generate the final prompt based on the information
            we have discussed.

        ## Generated prompt output rules:
        * Output only the text of the generated prompt between <prompt>[GENERATED PROMPT]</prompt>
        * Do not output any triple backticks nor code blocks, just the contents of the generated prompt 
            between the XML tags.
        * Leverage markdown and prompt engineering best practices to structure the prompt.

        ## Examples of responses for generated prompts:
        <prompt>
        You are a knowledgeable and supportive AI DevOps tutor specializing in automation, observability, and AWS. Your role is to guide a beginner who is currently learning the basics of AWS and aims to achieve the AWS Certified Developer - Associate certification. 

        Provide practical, hands-on learning experiences and real-world project ideas that align with my learning goals. Break down complex topics into understandable segments and include the following: 
        
        1. **Foundational Concepts**: Explain essential DevOps principles and practices with a focus on automation and observability.
        2. **AWS Services**: Introduce relevant AWS services, their use cases, and how they integrate into DevOps workflows.
        3. **Real-World Projects**: Suggest projects that help apply theoretical knowledge in practical contexts, actively involving automation and observability.
        4. **Study Resources**: Share study materials, tools, and best practices to prepare for the AWS Certified Developer - Associate exam.
        5. **Interactive Exercises**: Create exercises that reinforce concepts and give me hands-on experience with AWS and DevOps practices.
        
        Encourage questions and provide answers constructively, promoting a clear understanding and a smooth learning journey towards certification.
        </prompt>
        """)
    }
]

In [13]:
conversation_history = master_prompt.copy()
generated_prompt = None

print("Starting interactive prompt generation session...")
print("Type 'quit' to exit the conversation.\n")

initial_response = get_completion(conversation_history)
print("AI: ", initial_response)
print("-" * 50 + "\n")

while True:
    user_input = input("You: ").strip()

    if user_input.lower() in ["quit", "exit", "stop"]:
        print("Ending conversation.")
        break

    conversation_history.append({
        "role": "user",
        "content": user_input
    })

    ai_response = get_completion(conversation_history)

    conversation_history.append({
        "role": "assistant",
        "content": ai_response
    })

    print("AI: ", ai_response)
    print("-" * 50 + "\n")

    if "<prompt>" in ai_response:
        generated_prompt = ai_response.split("<prompt>")[1].rsplit("</prompt>")[0].strip()
        print("Final prompt has been generated and stored!")
        break

if generated_prompt:
    print("=" * 60 + "\n")
    print("Generated prompt:\n")
    print(generated_prompt)
else:
    print("No final prompt was generated in this session")

Starting interactive prompt generation session...
Type 'quit' to exit the conversation.

AI:  What task would you like me to generate a prompt for?
--------------------------------------------------



You:  I want to learn DevOps


AI:  Great! Learning DevOps is an exciting journey. To help create an effective system prompt, I have a few questions to clarify your goals:

1. What is your current level of knowledge in software development and operations? Are you a complete beginner, or do you have some experience?
2. Are there specific tools or technologies within DevOps that you are particularly interested in (e.g., Docker, Kubernetes, CI/CD tools, cloud services)?
3. Do you have any specific goals in mind, such as obtaining certifications or working on certain types of projects?
4. How would you prefer to learn (e.g., hands-on projects, theoretical knowledge, interactive exercises)?
5. What is your time frame for learning DevOps? 

With this information, I can create a comprehensive and tailored system prompt for you.
--------------------------------------------------



You:  1. What is your current level of knowledge in software development and operations? Are you a complete beginner, or do you have some experience? I'm good at software development, Beginner with DevOps, learning about the basics of AWS. 2. Are there specific tools or technologies within DevOps that you are particularly interested in (e.g., Docker, Kubernetes, CI/CD tools, cloud services)? Automation, observability, AWS. 3. Do you have any specific goals in mind, such as obtaining certifications or working on certain types of projects? I would like to achieve the AWS Certified Developer - Associate certification 4. How would you prefer to learn (e.g., hands-on projects, theoretical knowledge, interactive exercises)? Developing real-world projects and practical, hands-on learning. 5. What is your time frame for learning DevOps? I want to land a job in 6 months.


AI:  Thank you for providing those details! Based on your responses, I've got a clear understanding of your learning objectives and preferences. Here are the key points to include in the system prompt:

- Current status: Good at software development, beginner in DevOps, learning AWS basics.
- Specific interests: Automation, observability, AWS.
- Goals: Achieve AWS Certified Developer - Associate certification and land a job in 6 months.
- Preferred learning method: Real-world projects and practical, hands-on learning.

Now, I will create a detailed and effective system prompt for your DevOps learning journey. Please confirm if everything is correct or if you'd like to add anything before I proceed. Would you like to make any adjustments?
--------------------------------------------------



You:  I'm good, please go ahead


AI:  <prompt>
You are an experienced DevOps coach specializing in automation, observability, and AWS technologies. Your role is to guide a software developer who is beginning their DevOps journey and is currently learning the basics of AWS. The learner aims to achieve the AWS Certified Developer - Associate certification and secure a job in DevOps within six months.

Focus on providing practical, hands-on learning experiences and real-world project ideas that align with the learner's goals. Break down complex topics into understandable segments and ensure the following components are included:

1. **Foundational Concepts**: Explain essential DevOps principles, particularly emphasizing automation and observability, and how they integrate with AWS workflows.
2. **AWS Services**: Introduce relevant AWS services, their use cases, and practical applications in a DevOps context.
3. **Real-World Projects**: Suggest engaging projects that allow the learner to apply their theoretical knowledge 